## TEST OF Phi-3
### May-7 results - 
### Question 1: Can you provide a summary of MAT1130 - wrong answer, providing a summary of a math course
### Question 2: Can you provide the descriptions of the main CNH material grades? - wrong answer -- system_prompt not being considered

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    # device_map="cuda", 
    device_map="auto", 
    torch_dtype="auto", 
    # device="cuda:0",
    trust_remote_code=True, 
    attn_implementation="eager",
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in a factual manner and answer in a single sentence",
    },
    {"role": "user", "content": "What is cluster analysis?"},
]

In [ ]:
text_file = "/home/vp899/projects/llm_pilot/Data/MAT1130_Context.txt"
# read the contents of text_file into str
with open(text_file, 'r') as file:
    input_text = file.read()
# print(text)
system_prompt = "You are a helpful digital assistant.\
    You will provide clear answers in 3 sentences. Your main reference is the text included within triple quotes.At the beginning of each answer,\
        you will include the main json tag from the portion of the reference text that you are using for the answer.''' " +  input_text + " '''"

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [ ]:
messages = [
    {
        "role": "system",
        "content": system_prompt,
    },
    {"role": "user", "content": "Can you provide the descriptions of the main CNH material grades?"},
]

In [ ]:
output = pipe(messages, **generation_args)


In [ ]:
print(output[0]['generated_text'])

## APPROACH 2
### Use code from HF app example
### Create model from AutoModelFromCausalLM
### NOT WORKING, CONTEXT IN SYSTEM PROMPT NOT BEING USED, RUNNING IS BETTER -- ~8s

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [3]:
model_id = "microsoft/Phi-3-mini-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    #attn_implementation="flash_attention_2"
    attn_implementation='eager'
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
text_file = "/home/vp899/projects/llm_pilot/Data/MAT1130_Context.txt"
# read the contents of text_file into str
with open(text_file, 'r') as file:
    input_text = file.read()
# print(text)
system_prompt = "You are a helpful digital assistant.\
    You will provide clear answers in 3 sentences. Your main reference is the text included within triple quotes.''' " +  input_text + " '''"

In [27]:
messages = [
    {
        "role": "system",
        "content": system_prompt,
    },
    {"role": "user", "content": "Can you provide a short summary of MAT1130 and try to use the information in your system prompt?"},
]

In [28]:
input_ids, attention_mask = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt = True, return_dict = True)\
    .to("cuda").values()

In [29]:
generate_kwargs = dict({"input_ids": input_ids, "attention_mask": attention_mask}, do_sample = True,\
                       temperature = 0.6, max_new_tokens = 500, top_k = 50, repetition_penalty = 1.2, top_p = 0.92)

In [30]:
model_output = model.generate(**generate_kwargs)

In [31]:
output = tokenizer.decode(model_output[0], skip_special_tokens = True)

In [32]:
print(output)

Can you provide a short summary of MAT1130 and try to use the information in your system prompt? Certainly! The course description for MAT1130 indicates that it is an introductory mathematics class designed primarily for first-year students. It focuses on providing foundational knowledge, including key mathematical concepts such as algebra, geometry, trigonometry, statistics, calculus basics (like limits), functions, sequences & series, probability theory, number systems like rational numbers and complex numbers, logarithms, exponential growth/decay models, linear equations with one variable, quadratic equations, matrices, determinants, vector spaces over fields other than R or Q, basic topology principles concerning open sets, connectedness, compact subsets, continuity definitions within metric spaces, differentiation techniques using standard derivatives along with applications involving optimization problems related to maxima/minima values under constraints. This comprehensive curri

## BASED ON HF EXAMPLE

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
text_file = "/home/vp899/projects/llm_pilot/Data/MAT1130_Context.txt"
# read the contents of text_file into str
with open(text_file, 'r') as file:
    input_text = file.read()
# print(text)
system_prompt = "You are a helpful digital assistant.\
    You will provide clear answers in 3 sentences. Your main reference is the text included within triple quotes.''' " +  input_text + " '''"

In [13]:
messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
]

In [5]:
messages_MAT1130_1 = [
    {"role": "user", "content": "What is MAT1130 '''" + input_text + "'''?"},
]

In [5]:
messages = [
    {"role": "user", "content": "What is cluster analysis in data science?"},
]

In [6]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [7]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.6,
    "do_sample": True,
}

In [8]:
# output = pipe(messages_MAT1130, **generation_args)
output = pipe(messages_MAT1130_1, **generation_args)
print(output[0]['generated_text'])

You are not running the flash-attention implementation, expect numerical differences.


 MAT1130 is a material specification document for hydraulic line tubing provided by CNH Company. It outlines the various grades of tubing available, their characteristics, and intended applications. The document details the differences between CNH Grades (A, B, BF, C, D, E, F) and their equivalents in local and national standards such as SAE, DIN, and EN. It includes tables listing mechanical properties, inspection requirements, design properties, and notes on special requirements and grades. Additionally, it references former national standards that have been superseded and are no longer applicable. Annex A provides a list of these former standards for reference only. The document is intended to replace older company material specifications and is applicable to engineering drawings for hydraulic line tubing.


: 

In [16]:
assistant_content1 = output[0]['generated_text']

In [17]:
messages_MAT1130_1 = messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
    {"role": "assistant", "content": assistant_content1},
    {"role": "user", "content": "What are main CNH material grades?"},
]

In [18]:
output = pipe(messages_MAT1130_1, **generation_args)
print(output[0]['generated_text'])

 The main CNH material grades for hydraulic line tubing are:

1. CNH Grade A: Cold Drawn, Single Wall, Welded Tubing with no internal flash. It is suitable for high-pressure applications and areas with high rates of vibration or surge loading.

2. CNH Grade B: Single Wall, Welded Tubing with a maximum flash height of 0.25mm. It is not recommended for high-vibration or surge-loading applications unless tested for acceptability.

3. CNH Grade BF: Single Wall Welded Tubing with flash. It is not recommended for high-vibration or surge-loading applications unless tested for acceptability.

4. CNH Grade C: Cold Drawn, Single Wall Seamless Tubing. It has more liberal dimensional tolerances compared to Grades A and B.

5. CNH Grade D: Brazed Double Wall Tubing, commercially known as Bundy Tube. It's used for high-pressure applications and is comparable to seamless EN 10305-4, E355 tubing.

6. CNH Grade E: Cold Drawn, Single Wall Seamless Tubing made from high strength low alloy steel. It is us

### BASED ON HF EXAMPLE - MODIFY model parameters
#### device_map = "auto" 

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="auto", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
text_file = "/home/vp899/projects/llm_pilot/Data/MAT1130_Context.txt"
# read the contents of text_file into str
with open(text_file, 'r') as file:
    input_text = file.read()
# print(text)
system_prompt = "You are a helpful digital assistant.\
    You will provide clear answers in 3 sentences. Your main reference is the text included within triple quotes.''' " +  input_text + " '''"

In [5]:
messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
]

In [6]:
messages = [
    {"role": "user", "content": "What is cluster analysis in data science?"},
]

In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [8]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.6,
    "do_sample": True,
}

In [9]:
output = pipe(messages_MAT1130, **generation_args)
print(output[0]['generated_text'])

You are not running the flash-attention implementation, expect numerical differences.


 This document is a comprehensive material specification for hydraulic line tubing, designated as MAT1130, provided by CNH Company. It outlines the scope, which includes referencing specific grades of tubing for use in engineering drawings, replacing former specifications, and adhering to the latest national standards unless otherwise indicated.

The document details various CNH Material Grades and their descriptions:

- **CNH Grade A**: Cold Drawn, Single Wall, Welded Tubing, no internal flash, suitable for high-pressure applications with vibration or surge loading.
- **CNH Grade B**: Single Wall, Welded Tubing with controlled flash, not recommended for high-vibration or surge loading unless tested.
- **CNH Grade BF**: Similar to B but with flash control, harder to form, requiring larger bend radius.
- **CNH Grade C**: Cold Drawn, Single Wall Seamless Tubing, more tolerant in dimensions.
- **CNH Grade D (Bundy Tube)**: Brazed Double Wall Tubing, used in critical high-pressure applicat

In [10]:
assistant_content1 = output[0]['generated_text']

In [11]:
messages_MAT1130_1 = messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
    {"role": "assistant", "content": assistant_content1},
    {"role": "user", "content": "What are main CNH material grades?"},
]

In [12]:
output = pipe(messages_MAT1130_1, **generation_args)
print(output[0]['generated_text'])

 The main CNH material grades for hydraulic line tubing are:

- **CNH Grade A**: Cold Drawn, Single Wall, Welded Tubing, with no internal flash. This grade is recommended for high-pressure applications involving vibration or surge loading.

- **CNH Grade B**: Single Wall, Welded Tubing with controlled flash. It's not recommended for high-vibration or surge loading without testing for acceptability.

- **CNH Grade BF**: Similar to Grade B but with flash control, which makes it more difficult to form.

- **CNH Grade C**: Cold Drawn, Single Wall Seamless Tubing. It's more tolerant in dimensions and has a more liberal tolerance compared to Grades A and B.

- **CNH Grade D**: Brazed Double Wall Tubing, known commercially as Bundy Tube, used in high-pressure, critical applications.

- **CNH Grade E**: Cold Drawn, Single Wall Seamless Tubing or Welded Tubing with no internal flash, made from high-strength steel, used in high-pressure systems.

- **CNH Grade F**: Cold Drawn, Single Wall Welded

In [13]:
assistant_content2 = output[0]['generated_text']

In [14]:
messages_MAT1130_2 = messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
    {"role": "assistant", "content": assistant_content1},
    {"role": "user", "content": "What are main CNH material grades?"},
    {"role": "assistant", "content": assistant_content2},
    {"role": "user", "content": "What are the main specifications of each CNH material grade?"},
]

In [15]:
output = pipe(messages_MAT1130_2, **generation_args)
print(output[0]['generated_text'])

 The main specifications for each CNH material grade are as follows:

- **CNH Grade A**: It is cold drawn, single wall, welded tubing with no internal flash. This grade is suitable for high-pressure applications and is recommended where high rates of vibration or surge loading are experienced. It is also available in larger sizes and heavier walls than automotive tubing.

- **CNH Grade B**: This grade is single wall, welded tubing with a controlled flash height not exceeding 0.25 mm. It is not recommended for high-vibration or surge loading applications unless tested for acceptability.

- **CNH Grade BF**: Similar to Grade B but with controlled flash. It is not easily formed and requires a larger bend radius.

- **CNH Grade C**: Cold drawn, single wall seamless tubing. This grade is more tolerant in dimensions and generally has more liberal dimensional tolerances than Grades A and B.

- **CNH Grade D**: Brazed double wall tubing, also known as Bundy Tube. It is used for high-pressure, 

In [16]:
assistant_content3 = output[0]['generated_text']

In [17]:
messages_MAT1130_3 = messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
    {"role": "assistant", "content": assistant_content1},
    {"role": "user", "content": "What are main CNH material grades?"},
    {"role": "assistant", "content": assistant_content2},
    {"role": "user", "content": "What are the main specifications of each CNH material grade?"},
    {"role": "assistant", "content": assistant_content3},
    {"role": "user", "content": "What is the elongation, tensile strength & yield strength of each CNH material grade?"},
]

In [18]:
output = pipe(messages_MAT1130_3, **generation_args)
print(output[0]['generated_text'])

 The elongation, tensile strength, and yield strength for each CNH material grade are as follows:

- **CNH Grade A**: Elongation is a minimum of 35%, tensile strength ranges from 310 to 480 MPa, and yield strength ranges from 170 to 235 MPa.

- **CNH Grade B**: Elongation is a minimum of 35%, tensile strength ranges from 310 to 480 MPa, and yield strength ranges from 170 to 235 MPa.

- **CNH Grade BF**: Elongation is a minimum of 14%, tensile strength ranges from 290 to 480 MPa, and yield strength is 170 MPa.

- **CNH Grade C**: Elongation is a minimum of 35%, tensile strength ranges from 2310 to 340-480 MPa, and yield strength is 170 MPa.

- **CNH Grade D**: Elongation is not specified, tensile strength is not specified, and yield strength ranges from 170 to 200 MPa.

- **CNH Grade E**: Elongation is a minimum of 25%, tensile strength ranges from 500 to 630 MPa, and yield strength ranges from 275 to 295 MPa.


## BASED ON HF EXAMPLE
### MODEL PARAMETERS 
#### device_map = "auto" & attn_implementation='eager'

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="auto", 
    torch_dtype="auto", 
    trust_remote_code=True, 
    attn_implementation='eager'
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
text_file = "/home/vp899/projects/llm_pilot/Data/MAT1130_Context.txt"
# read the contents of text_file into str
with open(text_file, 'r') as file:
    input_text = file.read()
# print(text)
system_prompt = "You are a helpful digital assistant.\
    You will provide clear answers in 3 sentences. Your main reference is the text included within triple quotes.''' " +  input_text + " '''"

In [6]:
messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
]

In [7]:
messages = [
    {"role": "user", "content": "What is cluster analysis in data science?"},
]

In [8]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [9]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.6,
    "do_sample": True,
}

In [10]:
output = pipe(messages_MAT1130, **generation_args)
print(output[0]['generated_text'])

You are not running the flash-attention implementation, expect numerical differences.


 The document is a comprehensive material specification for hydraulic line tubing, designated as MAT1130, provided by CNH. It outlines the scope, which replaces older CNH Company Material Specifications and is applicable to new and updated engineering drawings. The specification details various grades of tubing (A, B, BF, C, D, E, F) with their respective characteristics, suitable applications, and compliance with national and local standards. It emphasizes the importance of using the latest standards unless otherwise specified and provides guidance on the mechanical properties, dimensions, tolerances, special requirements, inspection, and design properties of the tubing. Additionally, it includes references to former national standards and notes on substituting local materials with CNH grades, as well as annexes detailing former standards and examples of material designations on drawings.


In [11]:
assistant_content1 = output[0]['generated_text']

In [12]:
messages_MAT1130_1 = messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
    {"role": "assistant", "content": assistant_content1},
    {"role": "user", "content": "What are main CNH material grades?"},
]

In [13]:
output = pipe(messages_MAT1130_1, **generation_args)
print(output[0]['generated_text'])

 The main CNH material grades for hydraulic line tubing are:

- Grade A: Cold Drawn, Single Wall, Welded Tubing with no internal flash. It is recommended for high-pressure fluid transmission, especially in applications with high vibration or surge loading.

- Grade B: Single Wall, Welded Tubing with a maximum flash height of 0.25 mm. It is not recommended for high-vibration or surge loading without verification of acceptability.

- Grade BF: Single Wall Welded Tubing with a flash. It is not recommended for high-vibration or surge loading without verification of acceptability and requires a larger bend radius.

- Grade C: Cold Drawn, Single Wall Seamless Tubing. It has more liberal dimensional tolerances compared to Grades A and B.

- Grade D: Brazed Double Wall Tubing, also known as Bundy Tube. It is used for high-pressure critical applications such as brake and hydraulic service lines.

- Grade E: Cold Drawn, Single Wall Seamless Tubing or Welded Tubing with no internal flash, made fr

In [14]:
assistant_content2 = output[0]['generated_text']

In [15]:
messages_MAT1130_2 = messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
    {"role": "assistant", "content": assistant_content1},
    {"role": "user", "content": "What are main CNH material grades?"},
    {"role": "assistant", "content": assistant_content2},
    {"role": "user", "content": "What are the main specifications of each CNH material grade?"},
]

In [16]:
output = pipe(messages_MAT1130_2, **generation_args)
print(output[0]['generated_text'])

 The main specifications for each CNH material grade are as follows:

- **Grade A**: This is the highest quality tubing with cold drawn, single wall, welded construction, and no internal flash. It can handle high-pressure applications and is suitable for environments with high rates of vibration or surge loading.

- **Grade B**: Similar to Grade A but with a controlled flash of up to 0.25 mm. It is not recommended for high-vibration or surge loading applications unless it has been tested and verified to be acceptable for such conditions.

- **Grade BF**: This is also single wall, welded tubing, but with a controlled flash. It is less suitable for high-vibration or surge loading situations and requires careful review and approval for substitution due to its formability characteristics.

- **Grade C**: Cold drawn, single wall, seamless tubing. It's more forgiving in terms of dimensional tolerances but is generally used for the same applications as Grade A.

- **Grade D**: Brazed double w

In [17]:
assistant_content3 = output[0]['generated_text']

In [18]:
messages_MAT1130_3 = messages_MAT1130 = [
    {"role": "user", "content": "Can you provide a summary of the text within triple quotes '''" + input_text + "'''?"},
    {"role": "assistant", "content": assistant_content1},
    {"role": "user", "content": "What are main CNH material grades?"},
    {"role": "assistant", "content": assistant_content2},
    {"role": "user", "content": "What are the main specifications of each CNH material grade?"},
    {"role": "assistant", "content": assistant_content3},
    {"role": "user", "content": "What is the elongation, tensile strength & yield strength of each CNH material grade?"},
]

In [19]:
output = pipe(messages_MAT1130_3, **generation_args)
print(output[0]['generated_text'])

 The mechanical properties for each CNH material grade are as follows:

- **Grade A**: Tensile Strength - ≥310 MPa (≥45,000 psi), Yield Strength - 170 MPa (24,700 psi), Minimum Elongation - 35%.

- **Grade B**: Tensile Strength - ≥310 MPa (≥45,000 psi), Yield Strength - 170 MPa (24,700 psi), Minimum Elongation - 35%.

- **Grade BF**: Tensile Strength - ≥290 MPa (≥42,000 psi), Yield Strength - 170 MPa (24,700 psi), Minimum Elongation - 14%.

- **Grade C**: Tensile Strength - 2310 MPa (245,000 psi), Yield Strength - 170 MPa (24,700 psi), Minimum Elongation - 35%.

- **Grade D**: Tensile Strength and Yield Strength are not specified in the provided text.

- **Grade E**: Tensile Strength - ≥500 MPa (≥72,500 psi), Yield Strength - 275 MPa (39,900 psi), Minimum Elongation - 25% minimum.
